<a href="https://colab.research.google.com/github/NeurologiaLogic/Artificial-intelligence/blob/main/deepfashion_clothes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
#import data
!pip install --upgrade --no-cache-dir gdown
#images
!gdown 1U9YVXRJSYeoW1Jjxw_9CBI0BQUoWVbke
#information
!gdown 1sXKaWanOFuRzCYrodqYjmBTzCzh35FbY

In [ ]:
!unzip information.zip
!unzip img

In [ ]:
#data provided
abs_path = './information/Anno_coarse'
dataset_path = os.path.join(abs_path,"list_category_img.txt")
df = pd.read_table(dataset_path, sep="\t")
df[['img_path','category_label']] = df['image_name  category_label'].str.split(expand=True)
X = df["img_path"]
y= df["category_label"]
print(y.nunique())

In [ ]:
#model structure
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras import Model

#base model
base_model = EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    input_shape=(300,300,3)
)

#example
# inputs = tf.keras.Input(shape=(3,))
# x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
# outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

base_model_input = base_model.input
base_model_output = base_model.output
m = Flatten()(base_model_output)
m = Dense(200,activation=tf.nn.relu,name="D1")(m)
m = BatchNormalization()(m)
m = Dropout(0.01)(m)
m = Dense(100,activation=tf.nn.relu,name="D2")(m)
final_output_layer = Dense(46,activation=tf.nn.softmax)(m)
model = Model(inputs=base_model_input,outputs=final_output_layer)
model.summary()

In [ ]:
#freeze training
for model_struct in model.layers:
  if model_struct.name == "block7e_project_bn":
    break
  model_struct.trainable = False

In [ ]:
#checking trainable params after tweak
model.summary()

In [ ]:
#compiling model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),)

In [ ]:
#reshaping function 
from PIL import Image
from tensorflow.keras.utils import load_img
from numpy import asarray

def reshaping(img_path):
  return asarray(tf.keras.utils.load_img(img_path).resize((300,300)))

In [ ]:
#training model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
X_train = [reshaping(image) for image in X_train]
X_test = [reshaping(image) for image in X_test]
model.fit(X_train,y_train)

In [ ]:
#test reshaping images
# print(X_train[150])
# from PIL import Image
# from numpy import asarray

# img = asarray(Image.open('/content/img/1981_Graphic_Ringer_Tee/img_00000001.jpg'))
# print(img.shape,img.size)
# img = asarray(Image.open('/content/img/1981_Graphic_Ringer_Tee/img_00000001.jpg').resize((300,300)))
# print(img.shape,img.size)

In [ ]:
#Evaluate Model
from sklearn.metrics import accuracy_score
predicted = [model.predict(x_test) for x_test in X_test]
# Evaluating the model
acc_naive = accuracy_score(y_train,predicted)
print(f'model has {acc_naive.round(2)*100}% accuracy')

In [ ]:
#Save model
import joblib
joblib.dump(model,"deep_fashion_model.pkl")